In [2]:
%pip install numpy 
%pip install gym 
%pip install moviepy
%pip install tensorflow 
%pip install matplotlib
%pip install tdqm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1334 sha256=4b942101ec0acd9d3c473e0f8596161dd6d31041d1cf196dd1ee3be42cfa1487
  Stored in directory: c:\users\richa\appdata\local\pip\cache\wheels\af\02\71\aae0f7ee738abf19498353918ddae0f90a0d6ceb337b0bbc91
Successfully built tdqm
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import gym
import setuptools.dist
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

In [120]:
class DDQNAgent:
    def __init__(self, state_size, action_size):
        # Hyperparameters and misc
        self.n_actions = action_size
        self.lr = 0.05
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.005

        # Experience Replay
        self.memory_buffer_size = int(1e5)
        self.memory_buffer_pointer = 0
        self.memory_buffer_current_state = np.empty(shape=(self.memory_buffer_size, state_size))
        self.memory_buffer_next_state = np.empty(shape=(self.memory_buffer_size, state_size))
        self.memory_buffer_action = np.empty(shape=(self.memory_buffer_size,))
        self.memory_buffer_reward = np.empty(shape=(self.memory_buffer_size,))
        self.memory_buffer_done = np.empty(shape=(self.memory_buffer_size,))

        # Neural Networks 
        self.q_model = self.build_model(state_size, action_size)
        self.q_target_model = self.build_model(state_size, action_size)

    def build_model(self, state_size, action_size):
        model = Sequential([
            Input(shape=(state_size,)),
            Dense(units=48, activation='relu'),
            Dense(units=48, activation='relu'),
            Dense(units=action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.lr))
        return model
    
    def compute_action(self, current_state):
        if np.random.uniform(0,1) < self.epsilon:
            return np.random.choice(range(self.n_actions))
        else:
            q_values = self.q_model.predict(current_state)[0]
            return np.argmax(q_values)
        
    def store_episode(self, current_state, action, reward, next_state, done):
        self.memory_buffer_current_state[self.memory_buffer_pointer] = current_state
        self.memory_buffer_next_state[self.memory_buffer_pointer] = next_state
        self.memory_buffer_action[self.memory_buffer_pointer] = action
        self.memory_buffer_reward[self.memory_buffer_pointer] = reward
        self.memory_buffer_done[self.memory_buffer_pointer] = done
        
        self.memory_buffer_pointer += 1

        if self.memory_buffer_pointer >= self.memory_buffer_size:
            self.memory_buffer_pointer = 0

    def update_epsilon(self):
        self.epsilon = self.epsilon_decay * self.epsilon

    def train(self, batch_size):
        batch_indices = np.random.choice(self.memory_buffer_current_state.shape[0], batch_size)

        current_states = self.memory_buffer_current_state[batch_indices]
        rewards = self.memory_buffer_reward[batch_indices]
        done = self.memory_buffer_done[batch_indices]

        q_current_states = self.q_model.predict(current_states)
        q_current_states_target = self.q_target_model.predict(current_states)

        q_targets = rewards + (1 - done) * (self.gamma * np.amax(q_current_states_target, axis=1))
        action_indices = np.argmax(q_current_states_target, axis=1)

        for i in range(len(action_indices)):
            q_current_states[i][action_indices[i]] = q_targets[i]

        self.q_model.fit(current_states, q_current_states)

    def update_q_target_network(self):
        self.q_target_model.set_weights(self.q_model.get_weights())

In [121]:
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
n_episodes = 500
max_iterations_ep = 400
batch_size = 1000
q_target_update_freq = 10

agent = DDQNAgent(state_size, action_size)
total_steps = 0
n_training = 0

In [122]:
for episode in range(n_episodes):
    print(f"Episódio {episode}")
    current_state = env.reset()
    current_state = np.array([current_state[0]])

    rewards = 0

    for step in range(max_iterations_ep):
        total_steps += 1

        action = agent.compute_action(current_state)        
        next_state, reward, done, _, _ = env.step(action)
        next_state = np.array([next_state])

        rewards = rewards + reward
        agent.store_episode(current_state, action, reward, next_state, done)
        
        if done:
            agent.update_epsilon()
            break

        current_state = next_state
    
    print(" rewards: ", rewards)

    if total_steps >= batch_size:
        agent.train(batch_size=batch_size)
        n_training = n_training + 1
        if n_training % q_target_update_freq:
            agent.update_q_target_network()

Episódio 0
 rewards:  9.0
Episódio 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

d:\Projetos_Dev\IA\Q-learning\dqn-cart\.venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
 rewards:  10.0
Episódio 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
 rewards:  10.0
Episódio 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/s

In [123]:
def make_video():
    import time
    temp_env = gym.make('CartPole-v1', render_mode="rgb_array")
    env = gym.wrappers.RecordVideo(env=temp_env, video_folder='videos', name_prefix="test-video", episode_trigger=lambda x: x % 2 == 0)
    rewards = 0
    steps = 0
    done = False
    state = env.reset()
    state = np.array([state[0]])

    while not done:
        action = agent.compute_action(state)
        state, reward, done, _, _ = env.step(action)
        state = np.array([state])            
        steps += 1
        rewards += reward

    print(rewards)
    env.close()

make_video()

d:\Projetos_Dev\IA\Q-learning\dqn-cart\.venv\Lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Moviepy - Building video d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos\test-video-episode-0.mp4.
Moviepy - Writing video d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos\test-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos\test-video-episode-0.mp4
8.0
